<a href="https://colab.research.google.com/github/Davis1233798/ANN_Data/blob/main/rclone%20%2B%20aria2%20Webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Collection 合集
1. [connect_remote_storage.ipynb](https://colab.research.google.com/drive/1_Hg9suxwuwB2gALOxew72f1ZNcAXC0xb?usp=sharing)
2. [host_file_server.ipynb](https://colab.research.google.com/drive/1bjBiETbHdbDHb6IQSrEFu3-DwJmMPyzb?usp=sharing)
3. [goog/colab/keep-alive.user.js](https://raw.githubusercontent.com/foxe6/goog/main/userscripts/colab/keep-alive.user.js)
4. [host_ssh_server.ipynb](https://colab.research.google.com/drive/1502I-4RAV00tpOANWWyKPOdqS9U2SU7V?usp=sharing)
5. [run_browser_automation_tool.ipynb](https://colab.research.google.com/drive/13SWamKfY0-itUqd3SguFir02rVzt3HpK?usp=sharing)
6. [headless_browser_GUI.ipynb](https://colab.research.google.com/drive/1JRPFUX-p1IXOVkNLyhpdA0Ou8YjmaTCx?usp=sharing)
7. [remote_browser_automation.ipynb](https://colab.research.google.com/drive/14WeagkqECVmdPXWvPlj4nG2tj-wJZX_8?usp=sharing)
8. [remote_control_a_browser.ipynb](https://colab.research.google.com/drive/1UFkStnKIKEsbxUun8y8w9Swce3A6uiIj?usp=sharing)
9. [downloader_aria2_rpc.ipynb](https://colab.research.google.com/drive/17OhdKYDYfsa8frkOV3WxbwFZ2LdzTiYP?usp=sharing)



In [ ]:
#@title { display-mode: "form" }
#@markdown -Install Prerequisites 安装先决条件
import IPython
from IPython.display import clear_output

!python3.7 -m pip install goog aria2rpc==1.0.0 --use-deprecated=legacy-resolver -U --no-cache
clear_output()

In [ ]:
#@markdown Install aria2 安装 aria2
!pkill aria2c
!pkill cloudflared
import goog.colab
from mixlab import (
    runSh,
    PortForward_wrapper,
    findProcess,
    findPackageR
)
import os
import pathlib
import zipfile
import random
import re
import urllib.request
import requests
from urllib.parse import urlparse
import IPython
from IPython.display import clear_output

runSh('apt install -y aria2')
pathlib.Path('ariang').mkdir(mode=0o777, exist_ok=True)
if not os.path.exists("ariang/index.html"):
    urllib.request.urlretrieve(
        findPackageR('mayswind/AriaNg', 'AllInOne.zip'),
        'ariang/new.zip'
    )
    with zipfile.ZipFile('ariang/new.zip', 'r') as zip_ref:
        zip_ref.extractall('ariang')
    try:
        pathlib.Path('ariang/new.zip').unlink()
    except FileNotFoundError:
        pass
clear_output()

In [ ]:
#@title { display-mode: "form" }
#@markdown Start aria2 启动 aria2
!pkill aria2c
!pkill cloudflared
import goog.colab
from mixlab import (
    runSh,
    PortForward_wrapper,
    findProcess,
    findPackageR
)
import os
import pathlib
import zipfile
import random
import re
import urllib.request
import requests
from urllib.parse import urlparse
import IPython
from IPython.display import clear_output


save_location="/mnt/goog" #@param{type:"string"}
rpc_listen_port=6800
rpc_ip_host = None
on_download_complete="https://raw.githubusercontent.com/foxe6/MiXLab/master/resources/aria2/on_download_complete.py" #@param{type:"string"}
try:
    odc = requests.get(on_download_complete).text
    on_download_complete = "/content/"+os.path.basename(on_download_complete)
    open(on_download_complete, "wb").write(odc.encode())
    pathlib.Path(on_download_complete).chmod(0o777)
except:
    on_download_complete = None
pathlib.Path(save_location).mkdir(mode=0o777, exist_ok=True)
if not findProcess("aria2c", "--enable-rpc"):
        try:
            trackers = requests.get(
                "https://trackerslist.com/best_aria2.txt"
            ).text
            cmdC = (r"aria2c --enable-rpc --rpc-listen-port={} -D ".format(rpc_listen_port) +
                r"-d {} ".format(save_location) +
                r"-c " +
                fr"--bt-tracker={trackers} " +
                r"--bt-request-peer-speed-limit=8M " +
                r"--bt-max-peers=0 " +
                r"--seed-ratio=0.0 " +
                r"--max-upload-limit=2M " +
                r"--max-download-limit=8M " +
                r"--max-overall-download-limit=15M " +
                r"--max-connection-per-server=8 " +
                r"--max-download-result=100 " +
                r"--max-concurrent-downloads=1 " +
                r"--min-split-size=10M " +
                r"--follow-torrent=false " +
                r"--disable-ipv6=true " +
                r"--rpc-allow-origin-all=true " +
                r"--check-certificate=false " +
                r"--user-agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36' "+
                r"--peer-agent=Transmission/2.77 " +
                r"--peer-id-prefix=-TR2770- " +
                r"--split=20 " +
                (r"--on-download-complete='{}'".format(on_download_complete) if on_download_complete else "") +
                r" &")
            # print(cmdC)
            runSh(cmdC, shell=True)
            Server = PortForward_wrapper(
                "argotunnel",
                "",
                True,
                [['aria2_rpc', rpc_listen_port, 'http']],
                "jp",
                ["", random.randint(5000,5500)]
            )
            data = Server.start('aria2_rpc', displayB=False, v=False)
            rpc_ip_host = urlparse(data['url']).hostname
        except:
            print("aria2 RPC cannot be started! Please try to re-run the cell.")


clear_output()


In [ ]:
#@title { display-mode: "form" }
#@markdown - Start Web UI 启动网页界面
import goog.colab
from mixlab import (
    runSh,
    PortForward_wrapper,
    findProcess,
    findPackageR
)
import os
import pathlib
import zipfile
import random
import re
import urllib.request
import requests
from urllib.parse import urlparse


web_ui_port=6801
if not rpc_ip_host:
    raise ValueError("rpc_ip_host")
filePath = 'ariang/index.html'
with open(filePath, 'r+') as f:
    read_data = f.read()
    f.seek(0)
    f.truncate(0)
    read_data = re.sub(r'(rpcHost:"\w+.")|rpcHost:""', 'rpcHost:"{}"'.format(rpc_ip_host), read_data)
    read_data = re.sub(r'protocol:"\w+."', r'protocol:"ws"', read_data)
    read_data = re.sub(r'rpcPort:"\d+."', 'rpcPort:"80"', read_data)
    read_data = re.sub(r'globalStatRefreshInterval:1e3,downloadTaskRefreshInterval:1e3',
                        'globalStatRefreshInterval:5e3,downloadTaskRefreshInterval:5e3', read_data)
    f.write(read_data)
try:
    urllib.request.urlopen("http://localhost:{}".format(web_ui_port))
except:
    runSh("python3 -m http.server {} &".format(web_ui_port), shell=True, cd="ariang/")
Server = PortForward_wrapper(
    "argotunnel",
    "",
    True,
    [['ariang', web_ui_port, 'http']],
    "jp",
    ["", random.randint(5000,5500)]
)
data = Server.start('ariang', displayB=False, v=False)
print(urlparse(data['url'])._replace(scheme='http').geturl())



http://rocky-desire-sagem-anchor.trycloudflare.com


In [ ]:
# ============================= FORM ============================= #
#@markdown <font size=5>← Install rclone</font>
# ================================================================ #

import os
import IPython
from IPython.display import clear_output

!curl https://rclone.org/install.sh | sudo bash

try:
	os.makedirs("/root/.config/rclone", exist_ok=True)
except OSError as error:
	pass


clear_output()

In [ ]:
#@ ← title rclone config
import IPython
from IPython.display import clear_output

!rclone config
clear_output()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Mount/Unmount rclone Drive (Optional)</font>
# @markdown Mount a remote drive as a local drive on a mountpoint.
# @markdown ---
Cache_Directory = "DISK" #@param ["RAM", "DISK"]
# ================================================================ #

import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re

if not os.path.exists("/root/.ipython/mixlab.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://shirooo39.github.io/MiXLab/resources/mixlab.py \
                  -O /root/.ipython/mixlab.py"
  run(_spl(shellCmd))

from mixlab import (
    runSh,
    prepareSession,
    rcloneConfigurationPath,
)

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Available drive to mount/unmount:</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))

prepareSession()
content = open(f"{rcloneConfigurationPath}/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)

if Cache_Directory == 'RAM':
  cache_path = '/dev/shm'
elif Cache_Directory == 'DISK':
  os.makedirs('/tmp', exist_ok=True)
  cache_path = '/tmp'

def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    cmd = rf"rclone mount {mountNam.value}: {mPoint}" \
      rf" --config {rcloneConfigurationPath}/rclone.conf" \
      ' --user-agent "Mozilla"' \
      ' --buffer-size 256M' \
      ' --transfers 10' \
      ' --vfs-cache-mode full' \
      ' --vfs-cache-max-age 0h0m1s' \
      ' --vfs-cache-poll-interval 0m1s' \
      f' --cache-dir {cache_path}' \
      ' --allow-other' \
      ' --daemon'

    if runSh(cmd, shell=True) == 0:
      print(f"The drive have been successfully mounted! - \t{mPoint}")
    else:
      print(f"Failed to mount the drive! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"The drive have been successfully unmounted! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

The drive have been successfully mounted! - 	/content/drives/w


In [ ]:
# ==============================#============================== #
# @markdown <font size="4">Run this cell if there is no methods working in Method 1.</font>
# @markdown > If you want to run other cell, stop this first and then run it again if you need it.
# ==============================#============================== #

import time
from IPython.display import clear_output


def loop():
    counter = 0
    second = 60

    while True:
        clear_output()

        print(f'This cell has been running for {counter} minute(s)...')

        counter += 1
        time.sleep(second)


if __name__ == '__main__':
    try:
        loop()
    except KeyboardInterrupt:
        clear_output()
        exit
    else:
        pass


This cell has been running for 31 minute(s)...
